In [1]:
!nvidia-smi

Fri Dec 10 09:53:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   54C    P3    N/A /  N/A |    911MiB /  4040MiB |     32%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
import torch_geometric
from torch_geometric.datasets import ModelNet
from torch_geometric.transforms import SamplePoints
from torch_geometric.transforms import Compose
from torch_geometric.transforms import LinearTransformation
from torch_geometric.transforms import GenerateMeshNormals
from torch_geometric.transforms import NormalizeScale
from torch_geometric.loader import DataLoader
from torch_geometric.data import Batch
from torch_scatter import scatter_mean
import sys
import numpy as np

## NOTE:
# Made it work with ModelNet10 (in folder "data/ModelNet") that has only 10 classes
# Now I'll try to test it with ModelNet40 ("data/ModelNet40") with 40 classes

# ----------------------------------------------------------------
# Choosing device:
device = "cuda" if torch.cuda.is_available() else "cpu"

# ----------------------------------------------------------------
# Hyper parameters:
num_points = 50     # 1024 seems to be the limit...?
batch_size = 32      # not yet used
num_epochs = 20
learning_rate = 0.001

##########################################################################
#                  NO LONGER USED but leave them here...
F = [128, 512, 1024]  # Number of graph convolutional filters.
K = [6, 5, 3]  # Polynomial orders.
M = [512, 128, 10]  # Output dimensionality of fully connected layers.
##########################################################################

# ------------------------------------------------------------------------
## Data loading:
# For ModelNet10 change root to "data/ModelNet"   -> 10 classes
# For MpdelNet40 change root to "data/ModelNet40" -> 40 classes
# Don't forget to change accordingly the output layer from the model...
transforms = Compose([SamplePoints(num_points, include_normals=True), NormalizeScale()])
dataset_train = ModelNet(root="rgcnn_pytorch/data/ModelNet10", name='10', train=True, transform=transforms)
dataset_test = ModelNet(root="rgcnn_pytorch/data/ModelNet10", name='10', train=False, transform=transforms)

# Shuffle Data
dataset_train = dataset_train.shuffle()
dataset_test = dataset_test.shuffle()

# Verification...
print(f"Train dataset shape: {dataset_train}")
print(f"Test dataset shape:  {dataset_test}")

print(dataset_train[0])

Train dataset shape: ModelNet10(3991)
Test dataset shape:  ModelNet10(908)
Data(pos=[50, 3], y=[1], normal=[50, 3])


In [3]:
import torch.nn as nn
import torch_geometric.utils as utils
import torch_geometric.nn.conv as conv

class GetGraph(nn.Module):
    def __init__(self):
        """
        Creates the weighted adjacency matrix 'W'
        Taked directly from RGCNN

        input:
            point_cloud: shape = [batch_size * sample_number, n]
        output:
            adj_matrix:  shape = [batch_size, sample_number, sample_number]
        """
        super(GetGraph, self).__init__()

    def forward(self, point_cloud):
        total_points = point_cloud.shape[0]
        curr_batch_size = int(total_points/num_points)
        point_cloud = point_cloud.reshape(curr_batch_size, num_points, -1)

        point_cloud_transpose = point_cloud.permute(0, 2, 1)
        point_cloud_inner = torch.matmul(point_cloud, point_cloud_transpose)
        point_cloud_inner = -2 * point_cloud_inner
        point_cloud_square = torch.sum(torch.mul(point_cloud, point_cloud), dim=2, keepdim=True)
        point_cloud_square_tranpose = point_cloud_square.permute(0, 2, 1)
        adj_matrix = point_cloud_square + point_cloud_inner + point_cloud_square_tranpose
        adj_matrix = torch.exp(-adj_matrix)
        return adj_matrix


class GetLaplacian(nn.Module):
    def __init__(self, normalize=True):
        """
        Computes the Graph Laplacian from a Weighted Graph
        Taken directly from RGCNN - currently not used - might need to find alternatives in PyG for loss function
        """
        super(GetLaplacian, self).__init__()
        self.normalize = normalize

        def diag(self, mat):
        # input is batch x vertices
            d = []
            for vec in mat:
                d.append(torch.diag(vec))
            return torch.stack(d)

    def forward(self, adj_matrix):
        if self.normalize:
            D = torch.sum(adj_matrix, dim=1)
            eye = torch.ones_like(D)
            eye = self.diag(eye)
            D = 1 / torch.sqrt(D)
            D = self.diag(D)
            L = eye - torch.matmul(torch.matmul(D, adj_matrix), D)
        else:
            D = torch.sum(adj_matrix, dim=1)
            D = torch.diag(D)
            L = D - adj_matrix
        return L

In [4]:
import torch.nn as nn
import torch_geometric.utils as utils
import torch_geometric.nn.conv as conv

class RGCNN_model(nn.Module):
    def __init__(self, vertice, F, K, M, regularization = 0, dropout = 0):
        # verify the consistency w.r.t. the number of layers
        assert len(F) == len(K)
        super(RGCNN_model, self).__init__()
        '''
        F := List of Convolutional Layers dimensions
        K := List of Chebyshev polynomial degrees
        M := List of Fully Connected Layers dimenstions
        
        Currently the dimensions are 'hardcoded'
        '''
        self.F = F
        self.K = K
        self.M = M

        self.vertice = vertice
        self.regularization = regularization
        self.dropout = dropout

        # initialize the model layers
        self.get_graph = GetGraph()
        self.get_laplacian = GetLaplacian(normalize=True)
        self.pool = nn.MaxPool1d(vertice)
        self.relu = nn.ReLU()

        ###################################################################
        #                                 CHANGE HERE 
        self.conv1 = conv.ChebConv(6, 128, 6)
        self.conv2 = conv.ChebConv(128, 512, 5)
        self.conv3 = conv.ChebConv(512, 1024, 3)

        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 40)
        ###################################################################
        '''
        for i in range(len(F)):
            if i == 0:
                layer = ChebConv(Fin=3, K=K[i], Fout=F[i])
            else:
                layer = ChebConv(Fin=F[i-1], K=K[i], Fout=F[i])
            setattr(self, 'gcn%d'%i, layer)
        for i in range(len(M)):
            if i==0:
                layer = nn.Linear(F[-1], M[i])
            else:
                layer = nn.Linear(M[i-1], M[i])
            setattr(self, 'fc%d'%i, layer)
        '''

    def batched_dense_to_sparse(self, W):
        '''
            Function to transform a batched graph matrix into sparse format
            input:
                dense graph matrix of size ()

            output: 
                edge_index
                enge_weight
        '''
        edges = [utils.dense_to_sparse(graph) for graph in W]
        edge_index = torch.zeros([32, 2, num_points * num_points])
        edge_weight = torch.zeros([32, 1, num_points * num_points])
        for i, edge in enumerate(edges):
            edge_index[i] = torch.tensor(edge[0])
            edge_weight[i] = torch.tensor(edge[1])
        edge_index = edge_index.reshape(2, 32 * num_points * num_points)
        edge_weight = edge_weight.reshape(32 * num_points * num_points)
        return edge_index, edge_weigh


    def forward(self, x, batch):
        # forward pass
        W   = self.get_graph(x.detach())  # we don't want to compute gradients when building the graph
        edges = [utils.dense_to_sparse(graph) for graph in W]
        edge_index = torch.zeros([32, 2, num_points*num_points]).to(device)
        edge_weight = torch.zeros([32, 1, num_points*num_points]).to(device)
        for i, edge in enumerate(edges):
            edge_index[i] = torch.tensor(edge[0]).to(device)
            edge_weight[i] = torch.tensor(edge[1]).to(device)
        edge_index = edge_index.reshape(2, 32 * num_points*num_points).to(device)
        edge_weight = edge_weight.reshape(32 * num_points*num_points).to(device)
        
        edge_index = torch.tensor(edge_index, dtype=torch.long).to(device)
        edge_weight = torch.tensor(edge_weight, dtype=torch.float).to(device)
        x = torch.tensor(x, dtype=torch.float).to(device)
        
        out = self.conv1(x, edge_index, edge_weight, batch=batch)
        out = self.relu(out)

        W   = self.get_graph(out.detach())
        edges = [utils.dense_to_sparse(graph) for graph in W]
        edge_index = torch.zeros([32, 2, num_points*num_points]).to(device)
        edge_weight = torch.zeros([32, 1, num_points*num_points]).to(device)
        for i, edge in enumerate(edges):
            edge_index[i] = torch.tensor(edge[0]).to(device)
            edge_weight[i] = torch.tensor(edge[1]).to(device)
        edge_index = edge_index.reshape(2, 32 * num_points*num_points).to(device)
        edge_weight = edge_weight.reshape(32 * num_points*num_points).to(device)
        
        edge_index = torch.tensor(edge_index, dtype=torch.long).to(device)
        edge_weight = torch.tensor(edge_weight, dtype=torch.float).to(device)
        out = torch.tensor(out, dtype=torch.float).to(device)

        out = self.conv2(out, edge_index, edge_weight, batch=batch)
        out = self.relu(out)

        W   = self.get_graph(out.detach())
        edges = [utils.dense_to_sparse(graph) for graph in W]
        edge_index = torch.zeros([32, 2, num_points*num_points]).to(device)
        edge_weight = torch.zeros([32, 1, num_points*num_points]).to(device)
        for i, edge in enumerate(edges):
            edge_index[i] = torch.tensor(edge[0]).to(device)
            edge_weight[i] = torch.tensor(edge[1]).to(device)
        edge_index = edge_index.reshape(2, 32 * num_points*num_points).to(device)
        edge_weight = edge_weight.reshape(32 * num_points*num_points).to(device)
        
        edge_index = torch.tensor(edge_index, dtype=torch.long).to(device)
        edge_weight = torch.tensor(edge_weight, dtype=torch.float).to(device)
        out = torch.tensor(out, dtype=torch.float).to(device)
        
        out = self.conv3(out, edge_index, edge_weight, batch=batch)
        out = self.relu(out)

        '''
        for i in range(len(self.F)):
            x = getattr(self, 'gcn%d'%i)(x, L)
            print(x)
            x = self.relu(x)
        '''
        # curr_batch_size = int(out.shape[0]/50)
        out = out.reshape([-1, 50, 1024])

        out = out.permute(0, 2, 1) # Transpose
        out = self.pool(out)
        out.squeeze_(2)

        out = self.fc1(out)
        out = self.relu(out)

        out = self.fc2(out)
        out = self.relu(out)

        out = self.fc3(out)
        return out

        '''
        for i in range(len(self.M)):
            x = getattr(self, "fc%d"%i)(x)
        return x
        '''

model = RGCNN_model(num_points, F, K, M)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
conv1.bias 	 torch.Size([128])
conv1.lins.0.weight 	 torch.Size([128, 6])
conv1.lins.1.weight 	 torch.Size([128, 6])
conv1.lins.2.weight 	 torch.Size([128, 6])
conv1.lins.3.weight 	 torch.Size([128, 6])
conv1.lins.4.weight 	 torch.Size([128, 6])
conv1.lins.5.weight 	 torch.Size([128, 6])
conv2.bias 	 torch.Size([512])
conv2.lins.0.weight 	 torch.Size([512, 128])
conv2.lins.1.weight 	 torch.Size([512, 128])
conv2.lins.2.weight 	 torch.Size([512, 128])
conv2.lins.3.weight 	 torch.Size([512, 128])
conv2.lins.4.weight 	 torch.Size([512, 128])
conv3.bias 	 torch.Size([1024])
conv3.lins.0.weight 	 torch.Size([1024, 512])
conv3.lins.1.weight 	 torch.Size([1024, 512])
conv3.lins.2.weight 	 torch.Size([1024, 512])
fc1.weight 	 torch.Size([512, 1024])
fc1.bias 	 torch.Size([512])
fc2.weight 	 torch.Size([128, 512])
fc2.bias 	 torch.Size([128])
fc3.weight 	 torch.Size([40, 128])
fc3.bias 	 torch.Size([40])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.001, 'betas

In [14]:
# Training
loader_train = DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)

loss = torch.nn.CrossEntropyLoss()
correct_percentage_list = []
model.train()
for epoch in range(num_epochs):
    correct = 0
    for i, data in enumerate(loader_train):
        # make sure the gradients are empty
        optimizer.zero_grad()
        
        # Data preparation 
        pos = data.pos        # (num_points * 3)   
        normals = data.normal # (num_points * 3)
        x = torch.cat([pos, normals], dim=1)   # (num_points * 6)
        # x = x.unsqueeze(0)    # (1 * num_points * 6)     the first dimension may be used for batching?
        # x = x.type(torch.float32)  # other types of data may be unstable

        y = data.y              # (1)
        y = y.type(torch.long)  # required by the loss function
        
        x = x.to(device)      # to CUDA if available
        y = y.to(device)
     
        # Forward pass
        y_pred = model(x, data.batch)     # (32 * 40)
        class_pred = torch.argmax(y_pred.squeeze(0), dim=1)  # (1)  
        correct += int((class_pred == y).sum())       # to compute the accuracy for each epoch
        
        # loss and backward
        l = loss(y_pred, y)   # one value
        l.backward()          # update gradients
        
        # optimisation
        optimizer.step()
    
        if i%100==0:
            print(f"Epoch: {epoch}, Sample: {i}, Loss:{l}")
            print(f"Y predicted: {y_pred.shape}, Y label: {y.shape}")
            print(f"Predicted class: {class_pred}")
            print(f"Real Class:      {y}  ")
    print(f"~~~~~~~~~ CORRECT: {correct /(len(loader_train)* batch_size)} ~~~~~~~~~~~")
    correct_percentage_list.append(correct / len(dataset_train))
print(correct_percentage_list)

torch.save(model.state_dict(), "/home/alex/Alex_pyt_geom/models")

with torch.no_grad():
    model.eval()
    correct = 0
    for data in dataset_test:
        pos = data.pos        # (num_points * 3)   
        normals = data.normal # (num_points * 3)
        x = torch.cat([pos, normals], dim=1)   # (num_points * 6)
        x = x.unsqueeze(0)    # (1 * num_points * 6)     the first dimension may be used for batching?
        x = x.type(torch.float32)  # other types of data may be unstable

        y = data.y              # (1)
        y = y.type(torch.long)  # required by the loss function
        
        x = x.to(device)      # to CUDA if available
        y = y.to(device)
     
        # Forward pass
        y_pred = model(x)     # (1 * 40)

        class_pred = torch.argmax(y_pred)
        correct += int((class_pred == y).sum())

    print(f"Correct percentage : {correct / len(dataset_test)}")

/tmp/ipykernel_12232/2487681434.py:84: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index[i] = torch.tensor(edge[0]).to(device)
/tmp/ipykernel_12232/2487681434.py:85: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_weight[i] = torch.tensor(edge[1]).to(device)
/tmp/ipykernel_12232/2487681434.py:89: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(edge_index, dtype=torch.long).to(device)
/tmp/ipykernel_12232/2487681434.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.

Epoch: 0, Sample: 0, Loss:0.13235673308372498
Y predicted: torch.Size([32, 40]), Y label: torch.Size([32])
Predicted class: tensor([0, 6, 5, 7, 2, 9, 2, 2, 2, 3, 6, 9, 6, 7, 6, 2, 9, 8, 2, 1, 2, 5, 8, 2,
        5, 9, 2, 2, 0, 9, 9, 7], device='cuda:0')
Real Class:      tensor([0, 6, 5, 7, 2, 9, 2, 2, 2, 3, 6, 9, 1, 7, 6, 2, 9, 8, 2, 1, 2, 5, 8, 2,
        5, 9, 2, 2, 0, 9, 9, 7], device='cuda:0')  
Epoch: 0, Sample: 100, Loss:0.13669681549072266
Y predicted: torch.Size([32, 40]), Y label: torch.Size([32])
Predicted class: tensor([7, 7, 9, 5, 1, 2, 8, 1, 8, 7, 6, 9, 4, 9, 8, 3, 8, 9, 5, 8, 7, 2, 6, 7,
        5, 1, 9, 8, 6, 2, 3, 2], device='cuda:0')
Real Class:      tensor([7, 7, 9, 5, 1, 2, 8, 1, 8, 7, 6, 9, 4, 9, 8, 3, 8, 9, 5, 8, 7, 2, 4, 7,
        5, 1, 9, 8, 4, 2, 3, 2], device='cuda:0')  
~~~~~~~~~ CORRECT: 0.9285 ~~~~~~~~~~~
Epoch: 1, Sample: 0, Loss:0.048259370028972626
Y predicted: torch.Size([32, 40]), Y label: torch.Size([32])
Predicted class: tensor([8, 1, 2, 0, 7, 2, 7, 

KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    model.eval()
    correct = 0
    for data in dataset_test:
        pos = data.pos        # (num_points * 3)   
        normals = data.normal # (num_points * 3)
        x = torch.cat([pos, normals], dim=1)   # (num_points * 6)
        x = x.unsqueeze(0)    # (1 * num_points * 6)     the first dimension may be used for batching?
        x = x.type(torch.float32)  # other types of data may be unstable

        y = data.y              # (1)
        y = y.type(torch.long)  # required by the loss function
        
        x = x.to(device)      # to CUDA if available
        y = y.to(device)
     
        # Forward pass
        y_pred = model(x)     # (1 * 40)

        class_pred = torch.argmax(y_pred)
        # print("Pred: ", class_pred.item(), "Real: " , y.item())
        correct += int((class_pred == y).sum())

    print(f"Correct percentage : {correct / len(dataset_test)}")

In [ ]:
with torch.no_grad():
    model.eval()
    correct = 0
    for data in dataset_test:
        
        pos = data.pos        # (num_points * 3)   
        normals = data.normal # (num_points * 3)
        x = torch.cat([pos, normals], dim=1)   # (num_points * 6)
        x = x.unsqueeze(0)    # (1 * num_points * 6)     the first dimension may be used for batching?
        x = x.type(torch.float32)  # other types of data may be unstable

        y = data.y              # (1)
        y = y.type(torch.long)  # required by the loss function
        
        x = x.to(device)      # to CUDA if available
        y = y.to(device)
     
        # Forward pass
        y_pred = model(x)     # (1 * 40)

        class_pred = torch.argmax(y_pred)
        print("Pred: ", class_pred.item(), "Real: " , y.item())
        correct += int((class_pred == y_pred).sum())

    print(f"Correct percentage : {correct / len(dataset_test)}")
    

In [ ]:
with torch.no_grad():
    x = torch.rand([1, 1024, 6])*5+3
    x = x.to(device)
    y = model(x)
    y_class = torch.argmax(y)
    print(y)
    print(y_class)

WORK IN PROGRESS!!! 
Trying to create batches from data...

In [ ]:
import numpy as np
from torch_geometric.data import Data

length = len(dataset_train)
batch_size = 32
iterations = np.ceil(length/batch_size)
iterations = iterations.astype(int)
batched_data = torch.empty([125, 1024, 6])
print(dataset_train)
print(dataset_train[0])
aux = Data()
for i in range(iterations):
    ob = dataset_train[i:i+batch_size]
    pos=torch.empty([0, 3])
    y = torch.empty([0])
    normal = torch.empty([0, 3])
    for data in ob:
        pos = torch.cat([pos, data.pos])
        y = torch.cat([y, data.y])
        normal = torch.cat([normal, data.normal])
    batch_data[i] = Data(pos=pos, y=y, normal=normal)
print(pos.shape)

In [ ]:
#print(pos.shape)
#print(pos)
print(len(batch_data))
Batched_data = torch.empty([125, 1024, 6])
BATCHED_DATA = []
for i in range(125):
    # print(batch_data[i].pos)
    pos = torch.empty([32, 1024, 3])
    y = torch.empty([32, 1])
    normal = torch.empty([32, 1024, 3])
    for i in range(batch_size):
        pos[i] = batch_data[i].pos[num_points*i:num_points*i+1024]
        y[i] = batch_data[i].y[i]
        normal[i] = batch_data[i].normal[num_points*i:num_points*i+1024]
    BATCH = Data(pos=pos, y=y, normal=normal)
    BATCHED_DATA.append(BATCH)
    # Batched_data[i] = Data(pos=pos, y=y, normal=normal)
print(pos.shape)
print(normal.shape)
print(y.shape)
print(len(BATCHED_DATA))
for data in BATCHED_DATA:
    print(data)